# Decorator Sample 1

In [10]:
from dataclasses import dataclass
import datetime

def hide_field(field) -> str:
    return "**redacted**"

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")

def show_original(event_field):
    return event_field


class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields

    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation
            in self.serialization_fields.items()
        }

class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)

    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)

        event_class.serialize = serialize_method
        return event_class

@Serialization(
    username=str.lower,
    password=hide_field,
    ip=show_original,
    timestamp=format_time,
)
@dataclass
class LoginFormat:
    username: str
    password: str
    ip: str
    timestamp: datetime


In [12]:
from functools import wraps
def trace_decorator(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        #logger.info("running %s", function.__qualname__)
        #...
        return function(*args, **kwargs)

    return wrapped
    
@trace_decorator
def process_account(acount_id: str):
    """Process an account by ID. """
    #...

help(process_account)

Help on function process_account in module __main__:

process_account(acount_id: str)
    Process an account by ID.

